In [34]:
import pandas as pd
import numpy as np

In [35]:
df = pd.read_excel("C:\Data_Engineer_Study\멀캠_PJT_2\태블로 데이터\전체데이터관리.xlsx", sheet_name = "part 1")

In [36]:
df

,년/월,시도,산업_대분류,구인인원,구직건수,취업건수,사업체 수,실업급여 수급자 수,각 지역별 인구,GDP(실질),금리,CLI 지수,소비자 심리지수
0,201801,서울,"농업, 임업 및 어업",0,0,0,254,0,9851767,1812.0,1.50,100.37300,110.7
1,201801,서울,광업,0,0,0,25,0,9851767,1812.0,1.50,100.37300,110.7
2,201801,서울,제조업,33,0,1,32866,0,9851767,1812.0,1.50,100.37300,110.7
3,201801,서울,"전기, 가스, 증기 및 공기조절 공급업",1,0,0,96,0,9851767,1812.0,1.50,100.37300,110.7
4,201801,서울,"수도, 하수 및 폐기물 처리, 원료 재생업",0,0,0,396,0,9851767,1812.0,1.50,100.37300,110.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23455,202212,제주,"협회 및 단체, 수리 및 기타 개인 서비스업",0,0,0,1441,0,678159,1968.8,3.25,98.75089,90.2
23456,202212,제주,가구 내 고용활동 및 달리 분류되지 않은 자가소비 생산활동,0,0,0,0,0,678159,1968.8,3.25,98.75089,90.2
23457,202212,제주,국제 및 외국기관,0,0,0,1,0,678159,1968.8,3.25,98.75089,90.2
23458,202212,제주,분류불능,0,0,16,0,0,678159,1968.8,3.25,98.75089,90.2


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23460 entries, 0 to 23459
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년/월         23460 non-null  int64  
 1   시도          23460 non-null  object 
 2   산업_대분류      23460 non-null  object 
 3   구인인원        23460 non-null  int64  
 4   구직건수        23460 non-null  int64  
 5   취업건수        23460 non-null  int64  
 6   사업체 수       23460 non-null  int64  
 7   실업급여 수급자 수  23460 non-null  int64  
 8   각 지역별 인구    23460 non-null  int64  
 9   GDP(실질)     23460 non-null  float64
 10  금리          23460 non-null  float64
 11  CLI 지수      23460 non-null  float64
 12  소비자 심리지수    23460 non-null  float64
dtypes: float64(4), int64(7), object(2)
memory usage: 2.3+ MB


In [38]:
df.columns

Index(['년/월', '시도', '산업_대분류', '구인인원', '구직건수', '취업건수', '사업체 수', '실업급여 수급자 수',
       '각 지역별 인구', 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수'],
      dtype='object')

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_squared_log_error as msle


def get_scores_f(model, xtrain, xtest, ytrain, ytest):
    pred1 = model.predict(xtrain)   # train
    pred2 = model.predict(xtest)    # test

    # 음수가 있어도 상관없음
    A1 = r2_score(ytrain, pred1)
    A2 = r2_score(ytest, pred2)
    # msle, rmsle 는 음수가 있어서는 안됨
    pred2 = np.where(pred2<0, 0, pred2)
    B = msle(ytest, pred2)
    C = np.sqrt(B)   # rmsle
    data = [round(x, 4) for x in [A1, A2, C]]
    names = 'r2_train r2_test rmsle'.split()
    scores = pd.Series(data, index=names)
    return  scores

def make_models_f(xtrain, xtest, ytrain, ytest, n=300):
    temp = pd.DataFrame()

    model1 = LinearRegression().fit(xtrain, ytrain)
    temp['model1'] = get_scores_f(model1, xtrain, xtest, ytrain, ytest)

    model2 = DecisionTreeRegressor(random_state=0).fit(xtrain, ytrain)  # overfitting 나기 쉬움
    temp['model2'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model2 = DecisionTreeRegressor(max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model2_{d}'] = get_scores_f(model2, xtrain, xtest, ytrain, ytest)

    model3 = RandomForestRegressor(n, random_state=0).fit(xtrain, ytrain)
    temp['model3'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    for d in range(3, 9):
        model3 = RandomForestRegressor(n, max_depth=d, random_state=0).fit(xtrain, ytrain)
        temp[f'model3_{d}'] = get_scores_f(model3, xtrain, xtest, ytrain, ytest)

    model4 = XGBRegressor(objective='reg:squarederror').fit(xtrain, ytrain)
    temp['model4'] = get_scores_f(model4, xtrain, xtest, ytrain, ytest)

    temp = temp.T
    temp.insert(2, 'diff', (temp['r2_train'] - temp['r2_test']).abs())

    return temp

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23460 entries, 0 to 23459
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년/월         23460 non-null  int64  
 1   시도          23460 non-null  object 
 2   산업_대분류      23460 non-null  object 
 3   구인인원        23460 non-null  int64  
 4   구직건수        23460 non-null  int64  
 5   취업건수        23460 non-null  int64  
 6   사업체 수       23460 non-null  int64  
 7   실업급여 수급자 수  23460 non-null  int64  
 8   각 지역별 인구    23460 non-null  int64  
 9   GDP(실질)     23460 non-null  float64
 10  금리          23460 non-null  float64
 11  CLI 지수      23460 non-null  float64
 12  소비자 심리지수    23460 non-null  float64
dtypes: float64(4), int64(7), object(2)
memory usage: 2.3+ MB


In [41]:
# '시도', '산업_대분류' -> 카테고리형으로 형 변환

df['시도'] = df['시도'].astype('category').cat.codes
df['산업_대분류'] = df['산업_대분류'].astype('category').cat.codes

In [42]:
# 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수' -> 정규화 실시(값 범위 0~1)
scaler = MinMaxScaler()
df_scale = scaler.fit_transform(df[['GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수']])
df_scale
# '구직건수', '취업건수', '사업체 수', '실업급여 수급자 수', '각 지역별 인구' -> 표준화 실시(평균 0, 표준편차 1)

array([[0.        , 0.36363636, 0.4165242 , 1.        ],
       [0.        , 0.36363636, 0.4165242 , 1.        ],
       [0.        , 0.36363636, 0.4165242 , 1.        ],
       ...,
       [1.        , 1.        , 0.04024227, 0.45187166],
       [1.        , 1.        , 0.04024227, 0.45187166],
       [1.        , 1.        , 0.04024227, 0.45187166]])

In [43]:
# target 데이터인 '구인인원' 제외
# '년/월'의 경우 영향x -> 제외
df_X = df[['시도', '산업_대분류', '구직건수', '취업건수', '사업체 수', '실업급여 수급자 수',
       '각 지역별 인구', 'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수']]

# 타겟 데이터
df_Y = df[['구인인원']]

In [44]:
Xtrain = df_X.iloc[:16460]
Xtrain

,시도,산업_대분류,구직건수,취업건수,사업체 수,실업급여 수급자 수,각 지역별 인구,GDP(실질),금리,CLI 지수,소비자 심리지수
0,8,7,0,0,254,0,9851767,1812.0,1.5,100.3730,110.7
1,8,3,0,0,25,0,9851767,1812.0,1.5,100.3730,110.7
2,8,20,0,1,32866,0,9851767,1812.0,1.5,100.3730,110.7
3,8,17,0,0,96,0,9851767,1812.0,1.5,100.3730,110.7
4,8,13,0,0,396,0,9851767,1812.0,1.5,100.3730,110.7
...,...,...,...,...,...,...,...,...,...,...,...
16455,7,6,0,0,863,1,3361781,1918.7,0.5,102.5346,103.2
16456,7,10,0,0,7234,1,3361781,1918.7,0.5,102.5346,103.2
16457,7,18,0,0,6815,13,3361781,1918.7,0.5,102.5346,103.2
16458,7,12,0,0,4074,3,3361781,1918.7,0.5,102.5346,103.2


In [45]:
Ytrain = df_Y.iloc[:16460]
Ytrain

,구인인원
0,0
1,0
2,33
3,1
4,0
...,...
16455,1
16456,0
16457,18
16458,7


In [46]:
Xtest = df_X.iloc[16460:]
Xtest

,시도,산업_대분류,구직건수,취업건수,사업체 수,실업급여 수급자 수,각 지역별 인구,GDP(실질),금리,CLI 지수,소비자 심리지수
16460,7,4,0,0,0,0,3361781,1918.7,0.50,102.53460,103.2
16461,7,9,0,0,9734,2,3361781,1918.7,0.50,102.53460,103.2
16462,7,15,0,0,0,0,3361781,1918.7,0.50,102.53460,103.2
16463,7,22,0,0,4961,1,3361781,1918.7,0.50,102.53460,103.2
16464,7,0,0,0,0,0,3361781,1918.7,0.50,102.53460,103.2
...,...,...,...,...,...,...,...,...,...,...,...
23455,14,22,0,0,1441,0,678159,1968.8,3.25,98.75089,90.2
23456,14,0,0,0,0,0,678159,1968.8,3.25,98.75089,90.2
23457,14,5,0,0,1,0,678159,1968.8,3.25,98.75089,90.2
23458,14,11,0,16,0,0,678159,1968.8,3.25,98.75089,90.2


In [47]:
Ytest = df_Y.iloc[16460:]
Ytest

,구인인원
16460,0
16461,0
16462,0
16463,0
16464,0
...,...
23455,0
23456,0
23457,0
23458,0


In [48]:
Xtrain.columns

Index(['시도', '산업_대분류', '구직건수', '취업건수', '사업체 수', '실업급여 수급자 수', '각 지역별 인구',
       'GDP(실질)', '금리', 'CLI 지수', '소비자 심리지수'],
      dtype='object')

In [49]:
x_train, x_test, y_train, y_test = train_test_split(Xtrain, Ytrain,
                                                    test_size = 0.3,
                                                    random_state = 1)
print([x.shape for x in [x_train, x_test, y_train, y_test]])

[(11522, 11), (4938, 11), (11522, 1), (4938, 1)]


In [50]:
models = make_models_f(x_train, x_test, y_train, y_test)

c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Python38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimato

In [52]:
print(models.sort_values('rmsle').head(10))

          r2_train  r2_test    diff   rmsle
model3      0.9769   0.9256  0.0513  0.4286
model4      0.9968   0.9063  0.0905  0.4519
model2_8    0.9760   0.8509  0.1251  0.4856
model3_8    0.9570   0.9264  0.0306  0.4925
model2      1.0000   0.8318  0.1682  0.4993
model2_7    0.9674   0.8539  0.1135  0.5236
model3_7    0.9488   0.9260  0.0228  0.5331
model2_6    0.9552   0.8844  0.0708  0.5818
model3_6    0.9338   0.9247  0.0091  0.6028
model3_5    0.9048   0.9190  0.0142  0.7189
